In [1]:
import numpy  as np
import pandas as pd
import csv
import gradio as gr

# visualization libraries
import matplotlib.pyplot as plt
import plotly.express as px

import cv2
import torch
import torch.nn.functional as F

c:\Users\kings\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Live WebCam Demo Inception-V3

In [2]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 3)
state_dict = torch.load(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\models\Inception3_Adamproposed_augmediapipe.pt")

model.load_state_dict(state_dict)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

Using cache found in C:\Users\kings/.cache\torch\hub\pytorch_vision_v0.10.0


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [3]:
def predict_emotion(file_path):

    font_scale = 1.5
    font = cv2.FONT_HERSHEY_PLAIN
    
    cap = cv2.VideoCapture(file_path)
    if not cap.isOpened():
        raise IOError("Cannot open webcam")
    result_data = []
    
    while True:
        # Capture a frame
        ret, frame = cap.read()
        facedet = cv2.CascadeClassifier(r'C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\Msc-Project\haarcascades\haarcascade_frontalface_default.xml')
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = facedet.detectMultiScale(gray_frame, 1.1, 4)

        if len(faces) == 0:
            print("No faces detected in the frame")
            continue  # Skip processing this frame if no faces are detected

        for x, y, w, h in faces:
            roi_gray = gray_frame[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            facess = facedet.detectMultiScale(roi_gray)
            if len(facess) == 0:
                print("Face not detected")
            else:
                for (ex, ey, ew, eh) in facess:
                    face_roi = roi_color[ey:ey+eh, ex:ex+ew]  # cropping the face

        rgb_image = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2RGB)
        image = cv2.resize(rgb_image, (299, 299))
        image = image / 255.0  # normalization
        data = torch.from_numpy(image)
        data = data.type(torch.FloatTensor)
        data = data.to(device)
        reshaped_data = data.permute(2, 0, 1)  # Reshape the tensor to have 3 as the first dimension
        reshaped_data = reshaped_data.unsqueeze(0)  # add a fourth dimension
        outputs = model(reshaped_data)
        pred = F.softmax(outputs[0], dim=-1)
        final_pred = torch.argmax(pred, 0)

        if final_pred == 0:
            emotion = "Negative"
        elif final_pred == 1:
            emotion = "Neutral"
        else:
            emotion = "Positive"

        text_x, text_y = x, y - 10  # Adjust the position above the bounding box
        cv2.putText(frame, emotion, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

        conf, classes = torch.max(pred, -1)
        emotion_id = [0, 1, 2]
        class_dict = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
        class_names = [class_dict[label] for label in emotion_id]
        result_data.append({'Confidence':conf.item(),'max_confidence':pred.tolist(),'Emotion':class_names[classes.item()]})

        # # Write the result to a CSV file
        # with open('result.csv', mode='a', newline='') as file:
        #     writer = csv.writer(file)
        #     if file.tell() == 0:
        #         writer.writerow(['Confidence', 'max_confidence', 'Emotion'])  # Write header only if the file is empty
        #     writer.writerows(result_data)

        cv2.imshow("Facial Expression Recognition", frame)

        if cv2.waitKey(2) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return result_data


In [4]:
result = predict_emotion(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\datasets\CASME2\CASME2\CASME2-RAW\CASME2-RAW\sub01\EP19_06f.avi")

In [5]:
result

[{'Confidence': 0.9465426206588745,
  'max_confidence': [0.9465426206588745,
   0.050117842853069305,
   0.003339516930282116],
  'Emotion': 'Negative'},
 {'Confidence': 0.9426162838935852,
  'max_confidence': [0.9426162838935852,
   0.0537981316447258,
   0.003585565835237503],
  'Emotion': 'Negative'},
 {'Confidence': 0.9435549378395081,
  'max_confidence': [0.9435549378395081,
   0.052898358553647995,
   0.00354670244269073],
  'Emotion': 'Negative'},
 {'Confidence': 0.942773699760437,
  'max_confidence': [0.942773699760437,
   0.05353566259145737,
   0.003690673504024744],
  'Emotion': 'Negative'},
 {'Confidence': 0.9471259117126465,
  'max_confidence': [0.9471259117126465,
   0.04958953708410263,
   0.0032845649402588606],
  'Emotion': 'Negative'},
 {'Confidence': 0.945117712020874,
  'max_confidence': [0.945117712020874,
   0.05150369182229042,
   0.003378533525392413],
  'Emotion': 'Negative'},
 {'Confidence': 0.9467746615409851,
  'max_confidence': [0.9467746615409851,
   0.049

In [6]:
pd.DataFrame(result)

,Confidence,max_confidence,Emotion
0,0.946543,"[0.9465426206588745, 0.050117842853069305, 0.0...",Negative
1,0.942616,"[0.9426162838935852, 0.0537981316447258, 0.003...",Negative
2,0.943555,"[0.9435549378395081, 0.052898358553647995, 0.0...",Negative
3,0.942774,"[0.942773699760437, 0.05353566259145737, 0.003...",Negative
4,0.947126,"[0.9471259117126465, 0.04958953708410263, 0.00...",Negative
5,0.945118,"[0.945117712020874, 0.05150369182229042, 0.003...",Negative
6,0.946775,"[0.9467746615409851, 0.04988718777894974, 0.00...",Negative
7,0.943661,"[0.9436607360839844, 0.052546266466379166, 0.0...",Negative
8,0.941474,"[0.9414743781089783, 0.05484282970428467, 0.00...",Negative
9,0.940793,"[0.940792977809906, 0.05559122934937477, 0.003...",Negative


In [7]:
with gr.Blocks(gr.themes.Soft()) as demo:
    gr.Markdown(
    """
    Facial Expression Recognition
    """
    )

    # condition = gr.Radio(["New", "Used"], label="Vehicle Condition")
    video = gr.Video()
    generate_btn = gr.Button("Predict")


    generate_btn.click(fn=predict_emotion, inputs=video, outputs=)


# if __name__ == "__main__":
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\kings\miniconda3\lib\site-packages\gradio\routes.py", line 427, in run_predict
    output = await app.get_blocks().process_api(
  File "c:\Users\kings\miniconda3\lib\site-packages\gradio\blocks.py", line 1323, in process_api
    result = await self.call_function(
  File "c:\Users\kings\miniconda3\lib\site-packages\gradio\blocks.py", line 1051, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Users\kings\miniconda3\lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "c:\Users\kings\miniconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "c:\Users\kings\miniconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 807, in run
    result = context.run(func, *args)
  File "C:\Users\kings\AppData\Local\Temp\ipykernel_20828\3094263180.py", line 8, in predict_emotion


Face not detected
Face not detected
Face not detected
No faces detected in the frame
Face not detected
Face not detected
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
Face not detected
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
No faces detected in the frame
Face not detected
No faces detected in the frame
Face not detected
Face not detected
Face not detected


In [ ]:
with gr.Blocks(gr.themes.Soft()) as demo:
    gr.Markdown(
    """
    # Spotting facial expression in video sequences

    """)
     plot_type == "multiline":
        source = data.stocks()

        highlight = alt.selection(type='single', on='mouseover',
                                fields=['symbol'], nearest=True)

        base = alt.Chart(source).encode(
            x='date:T',
            y='price:Q',
            color='symbol:N'
        )

        points = base.mark_circle().encode(
            opacity=alt.value(0)
        ).add_selection(
            highlight
        ).properties(
            width=600
        )

        lines = base.mark_line().encode(
            size=alt.condition(~highlight, alt.value(1), alt.value(3))
        )

        return points + lines
    video = gr.Video(label='Upload your video here')
    generate_btn = gr.Button("Predict emotions")
    output = gr.Text(label="predicted emotions")

    generate_btn.click(fn=predict_emotion, inputs=[video], outputs=output)


# if __name__ == "__main__":
demo.launch()